## Notebook to download MotionTag data and upload it to the Database

*Created by Victoria Dahmen - Project Mobilität.Leben - last updated on 20th March 2024*

Follow the instructions (marked with **'TODO'**).


In [ ]:
# Overview:

# --- DOWNLOAD --------------------------------------------------------------
# Step 0: set up the environment i.e. all filepaths
# Step 1: download the WP and SL data (WP=waypoints, SL=storyline)       

# --- PROCESS ---------------------------------------------------------------
# Step 2: convert the gzip files to csv                                  
# Step 3: move .gz files from 'new' folder to 'gz' folder (as backups; much smaller than csv files)   
# Step 4: split the WP csv files into separate days                      

# --- UPLOAD ----------------------------------------------------------------
# Step 5: upload WP to PostgreSQL database, then delete the WP csv files                                                                                  
# Step 6: upload SL to PostgreSQL database, then delete the SL csv files                           
# Step 7: run processing pipeline

# ---------------------------------------------------------------------------

# Folder structure:

# MT_MoLe_tmp (root)
# ├── new_waypoints         (new downloaded waypoints - first as gz, then as csv, then deleted)
# ├── new_storylines        (new downloaded storylines - first as gz, then as csv,  then deleted)
# ├── gz_waypoints          (move waypoints gz files here as a backup)
# ├── gz_storylines         (move storylines gz files here as a backup)
# this file
# MotionTagConfigKey.txt
# C:\...\myDatabasePassword.txt

In [1]:
# SOME FUNCTIONS

import jwt
import requests
import datetime
import pandas as pd
import gzip
import os
import shutil
import psycopg2 # pip install psycopg2
import pandas as pd
import os
import csv

# not required for steps 1-4
def setUpDBConnection(user, myDBPasswordPath, host='XX.XXX.XX.XXX', dbname='mobilitaetleben'):
    # 1. set password
    with open(myDBPasswordPath) as f:
        mypassword = f.read()
    # 2. open connection
    try: 
        conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=mypassword)
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres database")
        print(e)
    # 3. make cursor
    conn.autocommit = True
    cursor = conn.cursor()
    return conn, cursor

### Step 0: Make sure this data is correct

In [3]:
# WP = waypoints, SL = storylines

# TODO: create folder "MT_MoLe_tmp" next to this file
# TODO: in "MT_MoLe_tmp", create subfolders "new_waypoints", "new_storylines", "gz_waypoints", "gz_storylines"
# TODO: update all the file paths below, as applicable

# set path to text file that contains the MotionTagConfigKey
MTConfigFile = r"MotionTagConfigKey.txt"

# path to save the downloaded files to)
path_WP_new = r"MT_MoLe_tmp\new_waypoints" 
path_SL_new = r"MT_MoLe_tmp\new_storylines" 

# path to save the backup files to
path_WP_gz = r"MT_MoLe_tmp\gz_waypoints" 
path_SL_gz = r"MT_MoLe_tmp\gz_storylines" 

# Database connection information
myDBPasswordPath = r"C:\Users\...\myDBPassword.txt"
myDBUsername = "postgres"

In [4]:
# get last storyline and waypoints date
conn, cursor = setUpDBConnection(myDBUsername, myDBPasswordPath)
query = "select max(started_on) from storyline;"
cursor.execute(query)
last_storyline_date = cursor.fetchone()[0]
query = "select max(tracked_at) from waypoints;"
cursor.execute(query)
last_waypoint_date = cursor.fetchone()[0]
print("last storyline date: ", last_storyline_date)
print("last waypoint date: ", last_waypoint_date)
print('\nNOTE: Use current last day as new first day for storyline (not for waypoints)')

last storyline date:  2024-01-18
last waypoint date:  2023-12-08 23:59:59

NOTE: Use current last day as new first day for storyline (not for waypoints)


### Step 1: Download data

In [ ]:
# WAYPOINT DATA
 
# TODO: set start and end date for the waypoints data
start_date = pd.to_datetime('2024-01-03') # included
end_date = pd.to_datetime('2024-01-20') # included
# TODO: number of days per file
offset = 1

def generateWaypointsFilenames(start_date,end_date,offset):
    fileURLs = []
    for sd in pd.date_range(start_date, end_date, freq=f'{offset}D'):
        # sample file name: Waypoints.TUM-ML.2023-06-01--2023-06-07.csv.gz
        filename = 'Waypoints.TUM-ML.' + str(sd.date()) + '--' + str((sd + pd.DateOffset(days=offset-1)).date()) + '.csv.gz'
        start_date = start_date + pd.DateOffset(days=offset)
        fileURLs.append(filename)
    return fileURLs

def downloadData(fileURLs,path,MTConfigFile):
    with open(MTConfigFile, 'r') as file:
        MotionTagConfigKey = file.read()
        mt_config = eval(MotionTagConfigKey)
    for filename in fileURLs:
        print(filename)
        jwt_token = jwt.encode({'sub': 'dumps', 'exp': datetime.datetime.now() + datetime.timedelta(minutes=15)},
                            mt_config['mt_secret'], algorithm="HS256")
        payload = {'jwt': jwt_token}
        r = requests.get(mt_config['mt_base_url'] + filename, params=payload)
        with open(path + '\\' + filename.split('/')[-1], 'wb') as file:
            file.write(r.content)

fileURLs = generateWaypointsFilenames(start_date,end_date,offset)
downloadData(fileURLs,path_WP_new,MTConfigFile)

In [5]:
# STORYLINE DATA

# TODO: set start and end date for the storyline data
start_date = pd.to_datetime('2023-11-29') # included 
end_date = pd.to_datetime('2024-01-10') # included
# TODO: number of days per file
offset = 1

def generateStorylineFilenames(start_date,end_date,offset):
    # generate URLs for downloading the files
    fileURLs = []
    for sd in pd.date_range(start_date, end_date, freq='D'):
        # sample file name: Storyline.TUM-ML.2023-06-01--2023-06-01.csv.gz
        filename = 'Storyline.TUM-ML.' + str(sd.date()) + '--' + str((sd + pd.DateOffset(days=offset-1)).date()) + '.csv.gz'
        start_date = start_date + pd.DateOffset(days=offset)
        fileURLs.append(filename)
    return fileURLs

fileURLs = generateStorylineFilenames(start_date,end_date,offset)
downloadData(fileURLs,path_SL_new,MTConfigFile)

Storyline.TUM-ML.2023-11-29--2023-11-29.csv.gz
Storyline.TUM-ML.2023-11-30--2023-11-30.csv.gz
Storyline.TUM-ML.2023-12-01--2023-12-01.csv.gz
Storyline.TUM-ML.2023-12-02--2023-12-02.csv.gz
Storyline.TUM-ML.2023-12-03--2023-12-03.csv.gz
Storyline.TUM-ML.2023-12-04--2023-12-04.csv.gz
Storyline.TUM-ML.2023-12-05--2023-12-05.csv.gz
Storyline.TUM-ML.2023-12-06--2023-12-06.csv.gz
Storyline.TUM-ML.2023-12-07--2023-12-07.csv.gz
Storyline.TUM-ML.2023-12-08--2023-12-08.csv.gz
Storyline.TUM-ML.2023-12-09--2023-12-09.csv.gz
Storyline.TUM-ML.2023-12-10--2023-12-10.csv.gz
Storyline.TUM-ML.2023-12-11--2023-12-11.csv.gz
Storyline.TUM-ML.2023-12-12--2023-12-12.csv.gz
Storyline.TUM-ML.2023-12-13--2023-12-13.csv.gz
Storyline.TUM-ML.2023-12-14--2023-12-14.csv.gz
Storyline.TUM-ML.2023-12-15--2023-12-15.csv.gz
Storyline.TUM-ML.2023-12-16--2023-12-16.csv.gz
Storyline.TUM-ML.2023-12-17--2023-12-17.csv.gz
Storyline.TUM-ML.2023-12-18--2023-12-18.csv.gz
Storyline.TUM-ML.2023-12-19--2023-12-19.csv.gz
Storyline.TUM

### Step 2: Convert .gz file to .csv file

In [ ]:
paths = [path_WP_new,path_SL_new] 

for path in paths:
    done = [f for f in os.listdir(path) if f[-4:]=='.csv']
    todo = [f for f in os.listdir(path) if (f[:-3] not in done) and (f[0]!='.') and (f[-3:]=='.gz')]
    for filename in todo:
        # avoid = '...'
        # if avoid in filename:
        #     continue
        print(filename)
        with gzip.open(os.path.join(path,filename), 'rt', newline='') as csv_file:
            csv_data = csv_file.read()
        newfilename = filename[:-3] # + '.csv'
        with open(os.path.join(path,newfilename), 'wt') as out_file:
            out_file.write(csv_data)

### Step 3: Move .gz files to 'waypoints' and 'storylines'

The gz files take up much less storage, so store these are a backup, instead of the csv files.

In [13]:
paths = [path_WP_new, path_SL_new] 
paths_for_gz_files = [path_WP_gz, path_SL_gz]  

for path,path_for_gz_file in zip(paths,paths_for_gz_files):
    for filename in os.listdir(path):
        if filename[-3:] == '.gz':
            shutil.move(os.path.join(path,filename), os.path.join(path_for_gz_file,filename))


### Step 4: Split waypoints files

In [14]:
# get all filenames in waypoints folder
filenames = [file for file in os.listdir(r'%s'%(path_WP_new))]

# distinguish between completed and new files
filenames_todo = [file for file in filenames if len(file)>31]
filenames_done = [file for file in filenames if len(file)==31]
print('Done:',len(filenames_done))
print('ToDo:',len(filenames_todo))

Done: 0
ToDo: 3


In [15]:
completed = [ i[17:-4] for i in filenames_done]
# TODO: these columns may (!) differ depending on the waypoints dataset
cols = ['user_id', 'tracked_at', 'latitude', 'longitude', 'accuracy', 'speed', 'altitude', 'course']

# reversed - to always get the most recent version of a day
print('Todo:',len(filenames_todo))
for filename in reversed(filenames_todo):
    print(filename)
    # load file as df
    df = pd.read_csv(r'%s\%s'%(path,filename),sep=';')
    # extract date as string
    df['date'] = df.tracked_at.apply(lambda x: x[:10])
    # remove dates that have already been dealt with
    df = df[~df['date'].isin(completed)]
    dates = df['date'].unique()
    # add dates to list of completed dates
    completed.extend(dates)
    #save each day as csv
    grouped = df.groupby('date')
    for date, group in grouped:
        print(date)
        group.to_csv(f'{path}/Waypoints.TUM-ML.{date}.csv', index=False, sep=';')

Todo: 3
Waypoints.TUM-ML.2024-01-11--2024-01-17.csv
2024-01-11
2024-01-12
2024-01-13
2024-01-14
2024-01-15
2024-01-16
2024-01-17
Waypoints.TUM-ML.2024-01-08--2024-01-10.csv
2024-01-08
2024-01-09
2024-01-10
Waypoints.TUM-ML.2024-01-05--2024-01-07.csv
2024-01-05
2024-01-06
2024-01-07


**STOP**

Make sure that no files (e.g. Waypoints.TUM-ML.2023-02-01.csv) are overwritten i.e. have 1KB size. --> Otherwise rerun the code for the file with the respective day using the code below.

In [ ]:
# TODO: check the sentence above.

# example: 
#       [[{date}, {corresponding file name}],\
#        ['2022-05-03', 'Waypoints.TUM-ML.2022-05-01--2022-05-07.csv.gz'],\
#        ['2022-07-04', 'Waypoints.TUM-ML.2022-07-02--2022-07-08.csv.gz']] 
to_redo = [] 

for filename in [i[1] for i in to_redo]:
    print(filename)
    # load file as df
    df = pd.read_csv(r'%s\%s'%(path,filename),sep=';')
    # extract date as string
    df['date'] = df.tracked_at.apply(lambda x: x[:10])
    # remove dates that have already been dealt with
    df = df[~df['date'].isin(completed)]
    dates = df['date'].unique()
    # add dates to list of completed dates
    completed.extend(dates)
    # save each day as csv
    grouped = df.groupby('date')
    for date, group in grouped:
        if date in [i[0] for i in to_redo]:
            print(date)
            group.to_csv(f'{path}/Waypoints.TUM-ML.{date}.csv', index=False, sep=';')

In [16]:
# Delete all filenames_todo (e.g., Waypoints.TUM-ML.2023-03-06--2023-03-12)
for filename in filenames_todo:
    os.remove(os.path.join(path,filename))


### Step 5: Upload waypoints

In [5]:
# connect to DB
if not 'cursor' in locals():
    conn,cursor = setUpDBConnection(myDBUsername,myDBPasswordPath)
    conn.autocommit = True

In [ ]:
# upload waypoints directly to the DB

filenames = [file for file in os.listdir(r'%s'%(path_WP_new)) if file[-4:]=='.csv']

with open(path_WP_new+r'\waypoints_uploaded.txt') as f:
    reader = csv.reader(f)
    try:
        completed = list(reader)[0]
    except:
        completed = []

filenames_todo = [i for i in filenames if i not in completed]

print('Files to be added:')
for file in filenames_todo:
    print(file)
    
# send command for each file to copy data to DB
for filename in filenames_todo:
    print(filename)
    try:
        copy_command = f'''COPY waypoints(user_id, tracked_at, latitude, longitude, accuracy, speed, altitude, course, date) 
                FROM STDIN DELIMITER ';' CSV HEADER;'''
        cursor.copy_expert(copy_command, open(r'%s\%s'%(path_WP_new,filename), "r"))
        print('V1')
    except:
        copy_command = f'''COPY waypoints(user_id, tracked_at, latitude, longitude, accuracy, speed, altitude, course) 
                FROM STDIN DELIMITER ',' CSV HEADER;'''
        cursor.copy_expert(copy_command, open(r'%s\%s'%(path_WP_new,filename), "r"))
        print('V2')
    completed.append(filename)
    # add date to list of completed dates
    with open(path_WP_new+r'\waypoints_uploaded.txt','w') as f:
        f.write(','.join(completed))
    conn.commit()


In [18]:
# Delete all csv files 
for filename in filenames:
    os.remove(os.path.join(path_WP_new,filename))


### Step 6: Upload storyline

Generate the commands to locally add the individual storyline files to the database. These can be collectively copied and pasted into the psql command window. Pretty fast: approx. 5 seconds per file

In [ ]:
import numpy as np
len_of_comment = []
# also check for all other days till end of november
for filename in os.listdir(path_SL_new):
    testdf = pd.read_csv(path_SL_new + r"\%s"%(filename), sep=';')
    if 'comment_feedback' in testdf.columns:    
        # max len and name of last column
        try: 
            max_len = max([len(i) for i in testdf.comment_feedback.unique() if i not in [np.nan,'nan']])
        except:
            max_len = 0
        len_of_comment.append(max_len)
print('Max length of comment:',max(len_of_comment))

In [ ]:
# STORYLINE

# \copy storyline(id, user_id, type, started_at, started_at_timezone, finished_at, finished_at_timezone, length, detected_mode, mode, purpose, geometry, confirmed_at, started_on, misdetected_completely, merged, created_at, updated_at,started_at_in_timezone,finished_at_in_timezone,confirmed_at_in_timezone,created_at_in_timezone,updated_at_in_timezone) FROM 'F:\Storyline.TUM-ML.2022-05-18--2022-05-31.csv' DELIMITER ';'CSV HEADER;

for filename in os.listdir(path_SL_new):
    if (filename[0]=='S') & (filename[-1]=='v'):
        filename = os.path.join(path,filename)
        print("\n\copy storyline(record_id, user_id, type, started_at, started_at_timezone, finished_at, finished_at_timezone, length, detected_mode, mode, purpose, geometry, confirmed_at, started_on, misdetected_completely, merged, created_at, updated_at,started_at_in_timezone,finished_at_in_timezone,confirmed_at_in_timezone,created_at_in_timezone,updated_at_in_timezone, comment_feedback) FROM '%s' DELIMITER ';'CSV HEADER;"%(filename))
print('\n')

**STOP**

**Warning:** make sure to not upload data that has previously been uploaded. If required, delete that data first. Deleting duplicates takes ages... 

Copy paste the above commands collectively into the "SQL Shell" (PSQL command terminal).
1. Open 'psql'
2. Enter server name: XX.XXX.XX.XXX
3. Enter db name: mobilitaetleben
4. Enter port: [press enter]
5. Enter username: [enter your username]
6. Enter password: [enter your password]
7. Paste the commands generated in the previous cell

In [34]:
# Delete all csv files 
for filename in os.listdir(path_SL_new):
    os.remove(os.path.join(path_SL_new,filename))

### Step 7: Run the processing pipeline for the new **storyline** data

In [ ]:
if not 'cursor' in locals():
       conn,cursor = setUpDBConnection(myDBUsername,myDBPasswordPath)

# get start and end date
manual_entry = False
if manual_entry:
       end_date = '20221231'
else:
       cursor.execute("SELECT MAX(started_on) FROM storyline;")
       end_date = cursor.fetchall()[0][0].strftime('%Y%m%d')
if manual_entry:
       start_date = '20220501'
else:
       cursor.execute("SELECT MAX(started_on) FROM storyline_processed;")
       start_date = cursor.fetchall()[0][0].strftime('%Y%m%d')
       # set start date to 2 days before because of overlapping trip ids
       start_date = pd.to_datetime(start_date) - pd.DateOffset(days=2)
       start_date = start_date.strftime('%Y%m%d')
# run pipeline.py where args=(query)
query = """select * from storyline
       where (started_on >= '%s') 
       AND (started_on < '%s');"""%(start_date,end_date)
print(query)

from pipeline import *
# result = runPipeline(query,write=True,mapMatching=False)

In [ ]:
# CHECK THAT THE NEW DATES ARE ADDED TO THE DB

conn,cursor = setUpDBConnection(myDBUsername,myDBPasswordPath)
queryX = "SELECT started_on FROM storyline_processed;"
cursor.execute(queryX)
data = cursor.fetchall()
set(data)

**NEXT**

Run pipeline_subtables.py

### Step 8: Make a backup of the database

In [ ]:
# ...